In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint as pprint

plt.style.use('seaborn')

In [58]:
df = pd.read_json('data/anime_content.json', lines=True)

In [59]:
df.head()

,_id,errors,data
0,{'$oid': '5ed06d2b64d930385059a26d'},"[{'message': 'Not Found.', 'status': 404, 'loc...",{'Media': None}
1,{'$oid': '5ed06d3564d930385059a26e'},NaN,"{'Media': {'id': 10161, 'title': {'romaji': 'N..."
2,{'$oid': '5ed06d3f64d930385059a26f'},NaN,"{'Media': {'id': 99726, 'title': {'romaji': 'N..."
3,{'$oid': '5ed06d4964d930385059a270'},NaN,"{'Media': {'id': 98526, 'title': {'romaji': 'R..."
4,{'$oid': '5ed06d5364d930385059a271'},NaN,"{'Media': {'id': 101240, 'title': {'romaji': '..."


In [60]:
# Some api calls returned None
no_media = []
for row in df['data']:
    if row['Media'] == None:
        no_media.append(True)
    else:
        no_media.append(False)

In [61]:
len(df[no_media])

48

In [62]:
df = df[~np.array(no_media)]

In [63]:
df[~np.array(df['errors'].isna())]

,_id,errors,data


In [64]:
# No remaining rows contain errors. Column is removed.
df = df[['_id', 'data']]

In [65]:
df.head()

,_id,data
1,{'$oid': '5ed06d3564d930385059a26e'},"{'Media': {'id': 10161, 'title': {'romaji': 'N..."
2,{'$oid': '5ed06d3f64d930385059a26f'},"{'Media': {'id': 99726, 'title': {'romaji': 'N..."
3,{'$oid': '5ed06d4964d930385059a270'},"{'Media': {'id': 98526, 'title': {'romaji': 'R..."
4,{'$oid': '5ed06d5364d930385059a271'},"{'Media': {'id': 101240, 'title': {'romaji': '..."
5,{'$oid': '5ed06d5e64d930385059a272'},"{'Media': {'id': 966, 'title': {'romaji': 'Cra..."


In [66]:
df.tail()

,_id,data
14473,{'$oid': '5ed2ab4364d930385059daf6'},"{'Media': {'id': 99916, 'title': {'romaji': 'A..."
14474,{'$oid': '5ed2ab4d64d930385059daf7'},"{'Media': {'id': 101283, 'title': {'romaji': '..."
14475,{'$oid': '5ed2ab5864d930385059daf8'},"{'Media': {'id': 101633, 'title': {'romaji': '..."
14476,{'$oid': '5ed2ab6264d930385059daf9'},"{'Media': {'id': 21742, 'title': {'romaji': 'K..."
14477,{'$oid': '5ed2ab6c64d930385059dafa'},"{'Media': {'id': 101089, 'title': {'romaji': '..."


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14430 entries, 1 to 14477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     14430 non-null  object
 1   data    14430 non-null  object
dtypes: object(2)
memory usage: 338.2+ KB


In [75]:
df['data'][1]['Media']

{'id': 10161,
 'title': {'romaji': 'No.6',
  'english': 'No.6',
  'native': 'NO.6 ナンバー・シックス',
  'userPreferred': 'No.6'},
 'startDate': {'year': 2011, 'month': 7, 'day': 8},
 'endDate': {'year': 2011, 'month': 9, 'day': 16},
 'season': 'SUMMER',
 'seasonYear': 2011,
 'type': 'ANIME',
 'format': 'TV',
 'status': 'FINISHED',
 'episodes': 11,
 'duration': 23,
 'chapters': None,
 'volumes': None,
 'isAdult': False,
 'genres': ['Action', 'Sci-Fi', 'Drama'],
 'tags': [{'name': 'Dystopian', 'rank': 92, 'category': 'Setting-Time'},
  {'name': "Boys' Love", 'rank': 70, 'category': 'Theme-Romance'},
  {'name': 'Shoujo', 'rank': 70, 'category': 'Demographic'},
  {'name': 'Cyberpunk', 'rank': 66, 'category': 'Theme-Sci-Fi'},
  {'name': 'Crossdressing', 'rank': 50, 'category': 'Cast-Traits'},
  {'name': 'Yandere', 'rank': 44, 'category': 'Cast-Traits'}],
 'isLicensed': True,
 'averageScore': 71,
 'popularity': 27608,
 'source': 'OTHER',
 'countryOfOrigin': 'JP',
 'staff': {'edges': [{'id': 12992,
 

In [120]:
# for show in df['data'][:5]:
#     print('anilist_id', show['Media']['id'])
#     print('title_romaji', show['Media']['title']['romaji'])
#     print('title_english', show['Media']['title']['english'])
#     print('title_native', show['Media']['title']['native'])
#     print('title_userPreferred', show['Media']['title']['userPreferred'])
#     print('start_date', pd.to_datetime(str(show['Media']['startDate']['year']) + '-' +
#                          str(show['Media']['startDate']['month']) + '-' +
#                          str(show['Media']['startDate']['day'])).date())
#     print('end_date', pd.to_datetime(str(show['Media']['endDate']['year']) + '-' +
#                          str(show['Media']['endDate']['month']) + '-' +
#                          str(show['Media']['endDate']['day'])).date())
#     print('season', show['Media']['season'])
#     print('type', show['Media']['type'])
#     print('format', show['Media']['format'])
#     print('status', show['Media']['status'])
#     print('episodes', show['Media']['episodes'])
#     print('duration', show['Media']['duration'])
#     print('chapters', show['Media']['chapters'])
#     print('volumes', show['Media']['volumes'])
#     print('isAdult', show['Media']['isAdult'])
#     print('genres', show['Media']['genres'])
#     print('type', show['Media']['tags'])

    
#     print('------------------')

anilist_id 10161
title_romaji No.6
title_english No.6
title_native NO.6 ナンバー・シックス
title_userPreferred No.6
start_date 2011-07-08
end_date 2011-09-16
season SUMMER
type ANIME
format TV
status FINISHED
episodes 11
duration 23
chapters None
------------------
anilist_id 99726
title_romaji Net-juu no Susume
title_english Recovery of an MMO Junkie
title_native ネト充のススメ
title_userPreferred Net-juu no Susume
start_date 2017-10-06
end_date 2017-12-08
season FALL
type ANIME
format TV
status FINISHED
episodes 10
duration 24
chapters None
------------------
anilist_id 98526
title_romaji Robomasters The Animated Series
title_english None
title_native ROBOMASTERS THE ANIMATED SERIES
title_userPreferred Robomasters The Animated Series
start_date 2017-10-13
end_date 2017-11-17
season FALL
type ANIME
format TV
status FINISHED
episodes 6
duration 25
chapters None
------------------
anilist_id 101240
title_romaji Dokidoki Little Ooya-san
title_english None
title_native dokidokiりとる大家さん
title_userPreferred

ParserError: Unknown string format: None-None-None

In [160]:
media_df = pd.DataFrame(list(df['data']))

In [161]:
media_df.head()

,Media
0,"{'id': 10161, 'title': {'romaji': 'No.6', 'eng..."
1,"{'id': 99726, 'title': {'romaji': 'Net-juu no ..."
2,"{'id': 98526, 'title': {'romaji': 'Robomasters..."
3,"{'id': 101240, 'title': {'romaji': 'Dokidoki L..."
4,"{'id': 966, 'title': {'romaji': 'Crayon Shin-c..."


In [162]:
media_df = pd.DataFrame(list(media_df['Media']))

In [163]:
media_df.head()

,id,title,startDate,endDate,season,seasonYear,type,format,status,episodes,...,genres,tags,isLicensed,averageScore,popularity,source,countryOfOrigin,staff,studios,characters
0,10161,"{'romaji': 'No.6', 'english': 'No.6', 'native'...","{'year': 2011, 'month': 7, 'day': 8}","{'year': 2011, 'month': 9, 'day': 16}",SUMMER,2011.0,ANIME,TV,FINISHED,11.0,...,"[Action, Sci-Fi, Drama]","[{'name': 'Dystopian', 'rank': 92, 'category':...",True,71.0,27608,OTHER,JP,"{'edges': [{'id': 12992, 'node': {'id': 101012...","{'edges': [{'id': 11804, 'node': {'id': 4, 'na...","{'edges': [{'id': 4370, 'role': 'SUPPORTING', ..."
1,99726,"{'romaji': 'Net-juu no Susume', 'english': 'Re...","{'year': 2017, 'month': 10, 'day': 6}","{'year': 2017, 'month': 12, 'day': 8}",FALL,2017.0,ANIME,TV,FINISHED,10.0,...,"[Comedy, Fantasy, Romance, Adventure, Slice of...","[{'name': 'Video Games', 'rank': 95, 'category...",True,75.0,41540,MANGA,JP,"{'edges': [{'id': 99978, 'node': {'id': 110641...","{'edges': [{'id': 17713, 'node': {'id': 6101, ...","{'edges': [{'id': 149462, 'role': 'MAIN', 'nod..."
2,98526,"{'romaji': 'Robomasters The Animated Series', ...","{'year': 2017, 'month': 10, 'day': 13}","{'year': 2017, 'month': 11, 'day': 17}",FALL,2017.0,ANIME,TV,FINISHED,6.0,...,"[Action, Drama]","[{'name': 'Real Robot', 'rank': 60, 'category'...",True,58.0,1901,ORIGINAL,JP,"{'edges': [{'id': 93476, 'node': {'id': 107517...","{'edges': [{'id': 17375, 'node': {'id': 6154, ...",{'edges': []}
3,101240,"{'romaji': 'Dokidoki Little Ooya-san', 'englis...","{'year': 2018, 'month': 5, 'day': 25}","{'year': 2019, 'month': 12, 'day': 27}",SPRING,2018.0,ANIME,OVA,FINISHED,6.0,...,[Hentai],"[{'name': 'Age Gap', 'rank': 86, 'category': '...",True,67.0,1228,VIDEO_GAME,JP,{'edges': []},"{'edges': [{'id': 18136, 'node': {'id': 6155, ...","{'edges': [{'id': 151698, 'role': 'MAIN', 'nod..."
4,966,"{'romaji': 'Crayon Shin-chan', 'english': 'Shi...","{'year': 1992, 'month': 4, 'day': 13}","{'year': None, 'month': None, 'day': None}",SPRING,1992.0,ANIME,TV,RELEASING,NaN,...,"[Comedy, Ecchi, Slice of Life]","[{'name': 'Episodic', 'rank': 85, 'category': ...",True,73.0,4681,MANGA,JP,"{'edges': [{'id': 966, 'node': {'id': 95148, '...","{'edges': [{'id': 2601, 'node': {'id': 55, 'na...","{'edges': [{'id': 2281, 'role': 'SUPPORTING', ..."
